In [1]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
from textblob import TextBlob

In [16]:
def get_SP500_stocks():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S.26P_500_Component_Stocks"
    r = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data, "html5lib")

    table = soup.find_all('table')[0]
    # Exclude header row
    rows = table.find_all('tr')[1:]
    #print(len(rows))
    #print(rows[len(rows)-1])
    stocks = []
    for row in rows:
        cols = row.find_all('td')
        stocks.append(cols[0].get_text())

    return stocks
sp500 = get_SP500_stocks() 

In [22]:
startdate = '2000-01-01'
enddate = '2017-11-01'
news = []
hm = ['hours ago', 'minutes ago', 'hour ago', 'minute ago']
news.append(('','',''))
f = open(' newsdata.txt', 'w')
for symbol in sp500:
    #if symbol == 'AAPL':
        i = 0
        fl = 1
        while fl == 1:
            url = 'https://finance.google.com/finance/company_news?q=' + symbol + '&startdate=' + startdate + \
                  '&enddate=' + enddate + '&start=' + str(i * 10) + '&num=10'
            x = requests.get(url)
            #print i
            #print url
            data = x.text
            soup = BeautifulSoup(data, 'html.parser')
            #print(soup.prettify())
            quote = soup.find_all('a', {"href" : lambda L: L and \
                    (L.startswith('http://news') or L.startswith('//www.google.com/url'))})
            if quote:
                for row in quote:
                    try:
                        item = row.parent.findNext('span',attrs={'class': 'date'}).next_element
                    except:
                        item = news[-1][2]
                        #news.append((symbol,row.next_element,news[-1][1]))
                    #print item
                    if any(x in str(item) for x in hm):
                        item = datetime.now().date()
                    else:
                        if item <> news[-1][2] and symbol == news[-1][0]:
                            item = datetime.strptime(item.replace(',',''), '%b %d %Y').date()
                    news.append((symbol,row.next_element.encode('ascii', 'ignore').decode('ascii')\
                    .replace('\r', '').replace('\n', ''),item))
                    line = row.next_element.encode('ascii', 'ignore').decode('ascii')\
                    .replace('\r', '').replace('\n', '')
                    f.write(symbol + '||' + str(line) + '||' + str(item) + '\n' + '||' \
                            + TextBlob(str(line)).polarity + '||' + TextBlob(str(line)).subjectivity)
            else:
                fl = 0
            i = i + 1
f.close()
#print 'Oldest Date news is:',min(dt for dt in [x[2] for x in news])
#print news 
print "end"

end
